In [2]:
from data import *

Using TensorFlow backend.


# Data Augmentasi

Pembuatan data augmentasi menggunakan library **keras.preprocessing.image.ImageDataGenerator** yang bisa menggabungkan Deep Neural Network image dan label secara bersamaan.

## Tentukan parameter data generator
Gunakan parameter dengan metode random dictionary seperti dibawah ini.

In [2]:
data_gen_args = dict()#rotation_range = 0.1,
                    #width_shift_range = 0.01,
                    #height_shift_range = 0.01,
                    #shear_range = 0.05,
                    #zoom_range = 0.1,
                    #horizontal_flip = True,
                    #fill_mode = 'nearest')
myGenerator = trainGenerator(30, 'data/train', 'image', 'label', data_gen_args, save_to_dir = "data/train/aug")

## Membuat data augmentasi ke folder **'data/train/aug/'**

In [3]:
num_batch = 20
for i, batch in enumerate(myGenerator):
    print(i)
    if(i>=(num_batch-1)):
        break

Found 600 images belonging to 1 classes.
Found 600 images belonging to 1 classes.
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


## Membuat .npy data (data augmentasi yang sudah dikompres)
Butuh memori tambahan untuk menyimpan data ini

In [4]:
image_arr,mask_arr = geneTrainNpy("data/train/aug/","data/train/aug/")
np.save("data/image_arr.npy",image_arr)
np.save("data/mask_arr.npy",mask_arr)

## Masking Test Data

In [16]:
import numpy as np 
import cv2

red = np.array([255, 0, 0])
green = np.array([0, 255, 0])
blue = np.array([0, 0, 255])
yellow = np.array([255, 255, 0])
aqua = np.array([0, 255, 255])

def masking(img, mask, color = None):
    mask_out = np.zeros((432, 532, 3), dtype = 'uint8')
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR) # kalau shapenya (432,532) atau grayscale
    for i in range(mask.shape[0]-1):
        for j in range(mask.shape[1]-1):
            if (mask[i,j] >= 220):
                mask_out[i,j,:] = mask[i,j] 
                mask_out[i,j,:] = color
                img[i,j,:] = mask_out[i,j,:]
    segmented = img
    return segmented

In [17]:
import os
from os import listdir
from skimage import io
from IPython.display import clear_output

results_path = 'data/test/'
results_results = 'results/'
list_file = os.listdir(results_path)
#real = np.zeros((len(list_file)//2, 432, 532), dtype ='uint8')
masked = np.zeros((len(list_file)//2, 432, 532, 3), dtype ='uint8')

for i in range(len(list_file)//2):
    masked[i] = masking(io.imread(results_path+str(i)+'.png'), io.imread(results_path+str(i)+'_predict.png'), yellow)

    if (i % 10 == 0):
        clear_output(wait=True)
    else:
        print('Creating... ' + results_path+str(i)+'_predict.png')

    io.imsave(os.path.join(results_results,"%d.png"%i), masked[i])

Creating... data/test/231_predict.png
Creating... data/test/232_predict.png
Creating... data/test/233_predict.png
Creating... data/test/234_predict.png
Creating... data/test/235_predict.png
Creating... data/test/236_predict.png
Creating... data/test/237_predict.png


# CARA CONVERT PNG TO MP4
## ffmpeg -r 30 -f image2 -i %d.png -vcodec libx264 -crf 15  -pix_fmt yuv420p test.mp4

# CARA MENGGABUNGKAN 2 VIDEO YANG SAMA
## ffmpeg -i Yoga60.avi -i aqua.mp4 -filter_complex "[0:v]pad=iw*2:ih[int]; [int][1:v]overlay=W/2:0[vid]" -map "[vid]" -c:v libx264 -crf 15 output2.mp4